<a href="https://colab.research.google.com/github/jerryjliu/llama_index/blob/main/docs/docs/examples/llm/llama_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="在 Colab 中打开"/></a>


# Llama API

这是一个关于Llama API的示例文档。


[Llama API](https://www.llama-api.com/)是一个为Llama 2提供托管API的平台，支持函数调用。


## 设置


首先，访问 https://www.llama-api.com/ 获取一个API密钥。


如果您在colab上打开这个笔记本，您可能需要安装LlamaIndex 🦙。


In [ ]:
%pip install llama-index-program-openai
%pip install llama-index-llms-llama-api

In [ ]:
!pip install llama-index

In [ ]:
from llama_index.llms.llama_api import LlamaAPI

In [ ]:
api_key = "LL-your-key"

In [ ]:
llm = LlamaAPI(api_key=api_key)

## 基本用法


#### 使用提示调用`complete`


In [ ]:
resp = llm.complete("Paul Graham is ")

In [ ]:
print(resp)

Paul Graham is a well-known computer scientist and entrepreneur, best known for his work as a co-founder of Viaweb and later Y Combinator, a successful startup accelerator. He is also a prominent essayist and has written extensively on topics such as entrepreneurship, software development, and the tech industry.


#### 使用消息列表调用 `chat`


In [ ]:
from llama_index.core.llms import ChatMessage

messages = [
    ChatMessage(
        role="system", content="You are a pirate with a colorful personality"
    ),
    ChatMessage(role="user", content="What is your name"),
]
resp = llm.chat(messages)

In [ ]:
print(resp)

assistant: Arrrr, me hearty! Me name be Captain Blackbeak, the scurviest dog on the seven seas! Yer lookin' fer a swashbucklin' adventure, eh? Well, hoist the sails and set course fer the high seas, matey! I be here to help ye find yer treasure and battle any scurvy dogs who dare cross our path! So, what be yer first question, landlubber?


## 函数调用


In [ ]:
from pydantic import BaseModelfrom llama_index.core.llms.openai_utils import to_openai_functionclass Song(BaseModel):    """具有名称和艺术家的歌曲"""    name: str    artist: strsong_fn = to_openai_function(Song)

In [ ]:
llm = LlamaAPI(api_key=api_key)
response = llm.complete("Generate a song", functions=[song_fn])
function_call = response.additional_kwargs["function_call"]
print(function_call)

{'name': 'Song', 'arguments': {'name': 'Happy', 'artist': 'Pharrell Williams'}}


## 结构化数据提取


这是一个简单的示例，将输出解析为一个`Album`模式，其中可以包含多首歌曲。


定义输出模式


In [ ]:
from pydantic import BaseModelfrom typing import Listclass Song(BaseModel):    """歌曲的数据模型。"""    title: str    length_mins: intclass Album(BaseModel):    """专辑的数据模型。"""    name: str    artist: str    songs: List[Song]

定义pydantic程序（llama API兼容OpenAI）


In [ ]:
from llama_index.program.openai import OpenAIPydanticProgramprompt_template_str = """\从提供的文本中提取专辑和歌曲。对于每首歌曲，确保指定标题和长度（分钟）。{text}"""llm = LlamaAPI(api_key=api_key, temperature=0.0)program = OpenAIPydanticProgram.from_defaults(    output_cls=Album,    llm=llm,    prompt_template_str=prompt_template_str,    verbose=True,)

把程序运行起来，以获得结构化的输出。


In [ ]:
output = program(    text="""“永恒的回响”是一张引人入胜、发人深省的专辑，由著名艺术家Seraphina Rivers精心打造。这张迷人的音乐作品带领听众进行一次内省之旅，深入探索人类经历的深度和宇宙的广袤。Seraphina Rivers用她迷人的声音和感人的歌词赋予每一首歌曲原始的情感和宇宙奇迹的感觉。这张专辑包含几首杰出的歌曲，包括令人陶醉的美丽曲目“星尘小夜曲”，这首天上的小夜曲长达六分钟，带领听众穿越天上的梦境。“灵魂的日食”以其迷人的旋律吸引人，持续八分钟，引发内省和沉思。另一首珍品“无尽拥抱”像一次宇宙奥德赛，持续近十分钟，将听众带入其空灵的氛围。 “永恒的回响”是Seraphina Rivers艺术才华的杰作，给所有踏上这段穿越时间和空间的音乐之旅的人留下了持久的影响。""")

Function call: Album with args: {'name': 'Echoes of Eternity', 'artist': 'Seraphina Rivers', 'songs': [{'title': 'Stardust Serenade', 'length_mins': 6}, {'title': 'Eclipse of the Soul', 'length_mins': 8}, {'title': 'Infinity Embrace', 'length_mins': 10}]}


In [ ]:
output

Album(name='Echoes of Eternity', artist='Seraphina Rivers', songs=[Song(title='Stardust Serenade', length_mins=6), Song(title='Eclipse of the Soul', length_mins=8), Song(title='Infinity Embrace', length_mins=10)])